In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import xlsxwriter

In [2]:
def get_page_meta(url):
    jar = requests.cookies.RequestsCookieJar()
    # 你可以維護一個糖果罐, 把不同網頁的 cookie 設定進來
    jar.set("over18", "1", domain="www.ptt.cc")
    if not "公告" in title and not "版規" in title:
        # 多帶入 cookies 參數
        response = requests.get(article_url, cookies=jar).text
        html = BeautifulSoup(response)
        content = html.find("div", id="main-content")
        # 準備我們要回傳的字典
        result = {}
        # 先刪選避免找不到或超過的list所產生的list index out of range
        try:
            values = content.find_all("span", class_="article-meta-value")
            # 先把文章資訊記錄在字典裡
            result["author"] = values[0].text
            result["board"] = values[1].text
            result["title"] = values[2].text
            result["time"] = values[3].text
            meta = content.find_all("div", class_="article-metaline")
            for m in meta:
                m.extract()
            right_meta = content.find_all("div", class_="article-metaline-right")
            for single_meta in right_meta:
                single_meta.extract()
            pushes = content.find_all("div", class_="push")
            score = 0
            for single_push in pushes:
                try:
                    pushtag = single_push.find("span", class_="push-tag").text
                    if "推" in pushtag:
                        score = score + 1
                    elif "噓" in pushtag:
                        score = score - 1
                    single_push.extract()
                except AttributeError:
                    print('沒分數')
                    pass
            # 分數和內容
            result["score"] = score
            result["content"] = content.text
            return result
        except IndexError:
            print("沒這東西，只好割捨")
        except AttributeError:
            print("沒這東西，割捨")
    # 公告和版規我就直接回傳 None
    else:
        return None

In [3]:
jar = requests.cookies.RequestsCookieJar()
jar.set("over18", "1", domain="www.ptt.cc")
# 從 MuscleBeach版首頁開始
url = "https://www.ptt.cc/bbs/MuscleBeach/index.html"
# 準備要記錄的表格
df = pd.DataFrame(columns=["author", "board", "title", "createdAt", "score", "content"])
# 走過五頁, range(5) 會幫你產生一個類似 [0, 1, 2, 3, 4] 的 list
try:
    for times in range(2359):
        response = requests.get(url, cookies=jar).text
        html = BeautifulSoup(response)
        # 得到每一篇文章的區域
        articles = html.find_all("div", class_="r-ent")
        # 走過每一篇文章
        for single_article in articles:
            # 得到 title 的超連結元素 <a>
            title_area = single_article.find("div", class_="title").find("a")
            # 如果有 title 才繼續 (被刪除的文章會沒有 title)
            if title_area:
                # 得到 title 的文字
                title = title_area.contents[0]
                # 得到 title 的超連結屬性href
                article_url = "https://www.ptt.cc" + title_area["href"]
                # 使用我們剛剛定義的函式
                print("第" + str(times) + '頁')
                print('title= ', title)
                result = get_page_meta(article_url)
                # 檢查是不是回傳 None(公告和版規會回傳 None)
                if result:
                    data = [result["author"], result["board"], result["title"], result["time"], result["score"],
                            result["content"]]
                    s = pd.Series(data, index=["author", "board", "title", "createdAt", "score", "content"])
                    df = df.append(s, ignore_index=True)
        time.sleep(5)
        # 往下一頁前進, string 參數可以找裡面文字符合我們帶入字串的元素
        url = "https://www.ptt.cc" + html.find("a", text=re.compile(r"上頁"))["href"]
    time.sleep(5)
except KeyError:
    print('已經沒了')

第0頁
title=  [贈送]  ON乳清空桶兩個（士林）
第0頁
title=  [問題] 中正中山區教練/健身房推薦
第0頁
title=  [閒聊] hypervolt plus台灣代理商訂價會不會太高
第0頁
title=  [閒聊] 免費運動按摩(已截止)
第0頁
title=  [問題] 舉重選手的胸
第0頁
title=  [徵求] 台北 乳清湯匙
第0頁
title=  [閒聊] 乳清一天喝的量有限制嗎
第0頁
title=  [問題] 台中WG Sport豐原店
第0頁
title=  [徵求] 乳清空桶 台南
第0頁
title=  [問題] 關於包裹內容物消失
第0頁
title=  [問題] 關於肩旋轉肌的放鬆的方法
第0頁
title=  [公告] 會藉、課程轉讓置底交流區
第0頁
title=  [公告] 肌肉沙灘板規 2019.5.27 版
第1頁
title=  [問題] 練到一半突然力量狂掉，甚至沒力
第1頁
title=  Re: [問題] 減脂期重訓訓練量的影響
第1頁
title=  [問題] Myprotein乳清蛋白分類選擇diet或isolate
第1頁
title=  [徵求] 輔大體育客乳清空桶
第1頁
title=  Re: [問題] 減脂卡關請益（成果報告）
第1頁
title=  [問題] 肌酸 會導致胃發炎嗎？
第1頁
title=  [問題] 請問有人可以分享hypervolt 使用心得嗎？
第1頁
title=  [心得] Cressey Performance體能訓練研習心得
第1頁
title=  [閒聊] 老外的運動裝扮
第1頁
title=  [新聞] 健身突發暈 努責現象引起 
第1頁
title=  [心得] 四年了 三項終於500
第1頁
title=  Re: [問題] 練到一半突然力量狂掉，甚至沒力
第1頁
title=  [問題] 請問有推薦WG光復店的教練嗎
第1頁
title=  [建議] 練呼吸如何改變你的人生
第1頁
title=  [問題] 重訓手套問題
第1頁
title=  [問題] 減脂期失眠如何改善
第2頁
title=  [問題] 想買按摩槍 二代hypervolt推嗎?
第2頁
title=  [問題] 深蹲後下背感覺肌痠
第2頁
title=  [心得] 鹹的

第14頁
title=  [情報] KAT 大專校隊訓練贊助計畫
第14頁
title=  [問題] 台中有推薦的專業健身教練嗎？
第14頁
title=  [徵求] myprotein高蛋白布朗尼求換
第14頁
title=  [問題] 尋求健力蹲舉裝備衣!!!!
第14頁
title=  [問題] 新店 新手 怎麼選？
第14頁
title=  [贈送] MP抹茶口味2.5KG X 2
第15頁
title=  Versa grips拉力帶 or 護腕選擇
第15頁
title=  Re: [問題] 胸、腰的贅皮？
第15頁
title=  [贈送] on雙巧
第15頁
title=  [徵求] 台中 徵求乳清空桶
第15頁
title=  [閒聊] 有人的cheat day 是完全不吃東西嗎
第15頁
title=  [問題] 臥推起槓歪斜（附圖）
第15頁
title=  [問題] 台中自由教練
第15頁
title=  [問題] 新店健工or信義健工
第15頁
title=  [問題] 健身工廠可以同時跟二個教練上課嗎？
第15頁
title=  [心得] 難民也想練身體？在家訓練10個月心得！
第15頁
title=  [贈送] 乳清空桶2個 基隆
第15頁
title=  Re: [閒聊] 最近很夯的網紅按摩槍和大陸按摩槍
第15頁
title=  [閒聊] 從Nano 8到Metcon 5
第15頁
title=  Re: [閒聊] 有人的cheat day 是完全不吃東西嗎
第15頁
title=  Re: [問題] 胸、腰的贅皮？
第15頁
title=  [問題] 娘娘蹲傷膝蓋？
第15頁
title=  [問題] 有上班族練法只吃乳清的肌肉照嗎
第15頁
title=  [情報] Myprotein限時52折
第15頁
title=  [問題] 巨巨朋友的困擾
第15頁
title=  [徵求] 彰化市一起運動好夥伴
第16頁
title=  [問題] myprotein買的到水解乳清嗎?
第16頁
title=  [討論] 「魔獸」豪爾德（Dwight Howard
第16頁
title=  [問題] 蹲舉腰(下背)閃到
第16頁
title=  [問題] 練前臂肩膀會痛
第16頁
title=  [問題] 高雄私人教練推薦
第16頁
ti

第27頁
title=  Re: [分享]倒立教學
第27頁
title=  [閒聊] 120+巨巨都穿什麼牌衣服
第27頁
title=  [徵求] 乳清空桶
第27頁
title=  [問題] 找sbd 衣服
第27頁
title=  [心得] 女生多件運動服心得(GYMSHARK.UA)
第27頁
title=  [問題] 在雙十WG握力帶被拿走
第28頁
title=  [閒聊] 健身房的停車優惠「高雄地區」
第28頁
title=  [閒聊] mars戰神布朗尼發霉
第28頁
title=  [閒聊] 你真心享受你現在的健身和飲食嗎?
第28頁
title=  [問題] 部位拆開練會有引響嗎
第28頁
title=  Re: [討論] 真的有人一週兩練然後練的不錯的嗎？
第28頁
title=  [問題] Jeff六分鐘痠痛訓練可以當作HIIT嗎
第28頁
title=  [討論] 平常外出會穿訓練鞋嗎
第28頁
title=  [問題]兩個小問題
第28頁
title=  [問題] 動畫健身少女教的觀念正確嗎
第28頁
title=  [問題] 槓轉動和護槓
第28頁
title=  [徵求] myp乳清巧克力薄荷換原味
第28頁
title=  [徵求] 徵求乳清空桶 士林天母
第28頁
title=  [心得] 竹竿人健身兩個月
第28頁
title=  Re: [問題] 深蹲腳踝會有移動感
第28頁
title=  [閒聊] 請問高雄有人可以教筋膜放鬆的嗎?
第28頁
title=  [閒聊] 健身房的月經文
第28頁
title=  [情報] Myprotein 限時51折
第28頁
title=  Re: [閒聊] 健身房的月經文
第28頁
title=  蘆洲三重健身房挑選
第28頁
title=  [問題] 抓力不夠
第29頁
title=  [徵求] 台南乳清空桶
第29頁
title=  [問題] 桃園健身房選擇
第29頁
title=  [贈送] 高雄鳳山乳清空桶(已贈完)
第29頁
title=  [問題] 一天其中一餐攝取過多蛋白質?
第29頁
title=  [問題] 新北三蘆/台北市徵私人教練
第29頁
title=  [心得] 壺鈴運動_關於手腕
第29頁
title=  [問題] 請問壓力褲 對於長時間站立工作
第29頁


第40頁
title=  [問題] 有推薦的健身書或健身教練嗎?
第40頁
title=  [贈送] 送乳清空桶一個高雄
第40頁
title=  [心得] 睪固酮檢驗和補充分享
第40頁
title=  [問題] 新店健工教練推薦
第40頁
title=  [問題] 手腕受傷，求助
第40頁
title=  [贈送] 長庚/龜山 5磅ON乳清空桶
第40頁
title=  [問題] 幾個特定肌肉的鍛鍊法
第41頁
title=  [閒聊] myprotein 衣服5折 補劑52折
第41頁
title=  [問題] 基隆健身房選擇(也可以幫忙推薦教練
第41頁
title=  [問題] 是否加入有氧（附上inbody)
第41頁
title=  [徵求] 乳清空桶 中壢
第41頁
title=  [心得] ACSM CPT 自學及考照心得
第41頁
title=  Re: [問題] 方便的雞胸肉取得方式
第41頁
title=  [閒聊] 體脂計令人崩潰
第41頁
title=  Re: [閒聊] 體脂計令人崩潰
第41頁
title=  Re: [閒聊] myprotein 衣服5折 補劑52折
第41頁
title=  [問題] 高雄健身房選擇
第41頁
title=  [贈送] 淡水乳清空桶5磅（送出）
第41頁
title=  [問題] 台南RONG RONG 一對一健身
第41頁
title=  Re: [心得] 有沒有不買教練課會被嗆的八卦(中和WG
第41頁
title=  [問題] 大家會對機械式器材有偏好嗎
第41頁
title=  [徵求] 乳清空桶-高雄（已徵到）
第41頁
title=  [情報] 讀者來信-關於教練課的大小事
第41頁
title=  [問題] 台中潭子豐原區推薦的教練
第41頁
title=  [問題] 肩膀軟骨破裂
第41頁
title=  [閒聊] 斷食運動Line群組歡迎大家加入
第41頁
title=  [問題] 碳水循環飲食請益
第42頁
title=  [問題] 啞鈴臥推上槓困難 導致肩膀不舒服
第42頁
title=  [心得] 成吉盃菁英盃三項賽事心得分享
第42頁
title=  [問題] 飲食問題(附上inbody)
第42頁
title=  [問題] 硬舉大屁屁
第42頁
title=  [問題] 

第53頁
title=  [閒聊] 黑私教？
第53頁
title=  [討論] 台灣街頭健身還有看頭嗎？
第53頁
title=  [問題] 健身房選擇
第53頁
title=  [問題] 高血壓是不是只能先有氧了
第53頁
title=  Re: [討論] 台鐵事件是不是練壯會有幫助?
第53頁
title=  [閒聊] 關於收器材
第53頁
title=  [徵求] 基隆基金 乳清空桶(小)
第54頁
title=  [問題] 震動花生球
第54頁
title=  [心得］Booster pro2 按摩槍
第54頁
title=  [徵求] 高雄健身咖
第54頁
title=  [贈送] ON乳清空桶一個（已送出）
第54頁
title=  [問題] 右上背部拉傷詢問
第54頁
title=  [心得] 恐怖硬拉腰椎受傷
第54頁
title=  [問題] 復健用護具 有勞巨巨們了
第54頁
title=  [問題] BCAA在斷食期間的使用時機
第54頁
title=  [贈送] 台中南區 ON金牌 5磅空桶 2罐(已送出)
第54頁
title=  [心得] 美國運動員訓練機構 P3 研習+參訪
第54頁
title=  [閒聊] world gym 很會漏水？
第54頁
title=  [閒聊] 紅牛乳清是不是有點小貴?
第54頁
title=  [徵求] 誠徵乳清５lb空桶
第54頁
title=  [問題] 請益板橋車站一帶的健身房
第54頁
title=  [討論] 健身算不算一個很燒錢的運動
第54頁
title=  Re: [討論] 健身算不算一個很燒錢的運動
第54頁
title=  [徵求] 送+換乳清（台北）已徵到謝謝
第54頁
title=  [問題] DROP SET的練法適合大肌群還是小肌群?
第54頁
title=  R:[問題] 胸、腰的贅皮？
第54頁
title=  Re: [討論] 健身算不算一個很燒錢的運動
第55頁
title=  [心得] WG會籍轉讓問題(已解決)
第55頁
title=  [徵求] (已徵到)乳清空桶
第55頁
title=  Re: [心得] 被Worldgym拒絕轉讓會籍
第55頁
title=  [問題] 全真瑜珈 租借置物櫃
第55頁
title=  [問題] 教練課刷退
第55頁
tit

...

第2304頁
title=  [問題] 左右手承受的強度不同
第2304頁
title=  Re: [閒聊]  inbody測量
第2304頁
title=  Re: [問題] 關於板橋蘇力士
第2304頁
title=  [問題] 請教有關仰臥起坐
第2304頁
title=  [心得] 健身會長高!?
第2304頁
title=  Re: [問題] 請教有關仰臥起坐
第2304頁
title=  [問題] 胸肌大小不一樣
第2305頁
title=  Re: [問題] 樹林哪裡有健身房比較推薦的
第2305頁
title=  [問題] 跳繩跳太久
第2305頁
title=  [閒聊] 二頭彎舉神人影片
第2305頁
title=  [已滿]中正運動中心 健身房回數券-行天宮/中正 …
第2305頁
title=  [問題] 關於板橋蘇力士
第2305頁
title=  [閒聊] 女子 口惡
第2305頁
title=  [請益] 配一些大重量??
第2305頁
title=  Re: [問題] 跳繩跳太久
第2305頁
title=  [問題] 請問我的姿勢正確嗎？
第2305頁
title=  [問題] 挑戰極限!?
第2305頁
title=  [問題] 肌肉和脂肪的組成方式
第2305頁
title=  Re: [心得] 台大的小叮噹
第2305頁
title=  [閒聊] 賣場的百貨部門
第2305頁
title=  [問題] 請問如何練股四頭肌內側的肌肉呢
第2305頁
title=  Re: [問題] 請問如何練股四頭肌內側的肌肉呢
第2305頁
title=  [問題] 請問所謂的"飛鳥"是否就是平舉?
第2305頁
title=  Re: [心得] 台大的小叮噹
第2305頁
title=  Re: [閒聊] 二頭彎舉神人影片
第2305頁
title=  [請益] 關於蹲舉
第2305頁
title=  [問題] 各位好 新手一個小問題問大家!
第2306頁
title=  [討論] 本版討論風氣
第2306頁
title=  Re: [閒聊] 關於「一般人」的定義
第2306頁
title=  [公告] 請注意發文/推文用詞
第2306頁
title=  [建議] 大家貼照片好了
第2306頁
title=  [情報] 一些名人的臥推戰鬥力

第2317頁
title=  Re: [問題] 身體太厚怎麼辦
第2317頁
title=  Re: [問題] 身體太厚怎麼辦
第2317頁
title=  Re: [問題] 身體太厚怎麼辦
第2317頁
title=  Re: [問題] 身體太厚怎麼辦
第2317頁
title=  [問題] 女生鍊胸大肌?
第2317頁
title=  [問題] 腹肌的鍛鍊
第2317頁
title=  [討論] 請大家給我健身重訓的理由
第2317頁
title=  [閒聊] 此風不可長
第2317頁
title=  Re: [討論] 請大家給我健身重訓的理由
第2317頁
title=  [問題] 請問有站著練胸肌的方法嗎?
第2317頁
title=  [問題] 半&反仰臥起坐姿勢
第2317頁
title=  [問題] 這個動作有用嗎？有害嗎？
第2318頁
title=  Re: [問題]我對我的課表有些疑問，想跟各位請教一下
第2318頁
title=  [閒聊] 磅數偷真大 冏
第2318頁
title=  Re: [討論] 卡特有在Gigger stronger faster 裡
第2318頁
title=  [新聞]舉重： 南韓靠人參提煉金牌
第2318頁
title=  Re: [討論] 卡特有在Gigger stronger faster 裡
第2318頁
title=  [問題] 已辦好的加州的會籍可以再更改嗎
第2318頁
title=  [問題] 練肌肉的方法
第2318頁
title=  Re: [閒聊] 磅數偷真大 冏
第2318頁
title=  Re: [閒聊] 磅數偷真大 冏
第2318頁
title=  [問題] 哪裡可以買得到便宜的健身手套？
第2318頁
title=  [問題] 南台科大的重訓室
第2318頁
title=  Re: [問題] 南台科大的重訓室
第2318頁
title=  [問題] 請問股四頭肌疼痛
第2318頁
title=  Re: [問題] 請問股四頭肌疼痛
第2318頁
title=  2008健身體適能動作示範影片in中山大學體適能室
第2318頁
title=  [問題] 兩個問題
第2318頁
title=  Re: [問題] 兩個問題
第2318頁
title=  [問題] 關於舉重床
第2318頁


第2329頁
title=  Re: [閒聊] 有點不高興!!
第2329頁
title=  [問題] 夏天到了想曬黑~
第2329頁
title=  [問題] 肩膀關節的問題
第2329頁
title=  [問題] 游泳或跑步那一種效果好?
第2329頁
title=  Re: [閒聊] 有點不高興!!
第2329頁
title=  [問題] 史密斯握推機
第2329頁
title=  Re: [問題] 肩膀關節的問題
第2330頁
title=  Re: [心得] 與交大健身房初次接觸
第2330頁
title=  [問題] 請問正面闊背?
第2330頁
title=  [問題] 仰臥起坐
第2330頁
title=  Re: [問題] 腹肌
第2330頁
title=  [問題] 逢甲健身房  校外人員加入?
第2330頁
title=  [問題] 中壢 奧大力亞  or不老林(新天地)
第2330頁
title=  [閒聊] 一些照片~~
第2330頁
title=  [問題] 關於測體脂
第2330頁
title=  [問題] 有人用過這種滾輪嗎?
第2330頁
title=  [問題] 買啞鈴問題
第2330頁
title=  Re: [閒聊] 瘦皮猴[徵]成大健身房同好
第2330頁
title=  [問題] 有人練胸肌練到抽筋嗎?
第2330頁
title=  [問題] 加州教練
第2330頁
title=  [問題] 體脂計價錢與建議
第2330頁
title=  [問題] 關於跑步的心跳
第2330頁
title=  [問題] 請問一下健身後痠痛?
第2330頁
title=  [討論] 為何走上健身這條路
第2330頁
title=  Re: [討論] 為何走上健身這條路
第2330頁
title=  [問題] 強力球 POWER BALL要去哪買啊？
第2330頁
title=  Re: [問題] 有人練胸肌練到抽筋嗎?
沒這東西，只好割捨
第2331頁
title=  Re: [問題] 鐵克網復活了嗎?
第2331頁
title=  Re: [問題] 周美青女士的上胸
第2331頁
title=  [問題] 想請問各位高手大大
第2331頁
title=  Re: [問題] 想請問各位高手大大
第2331頁
title=  [問題] 台北市

第2342頁
title=  Re: [問題] 每組訓練之間休息多久?
第2342頁
title=  [討論] 瘦子重訓適合大磅數訓練嗎
第2342頁
title=  [問題] 重訓疑惑
第2342頁
title=  Re: [討論] 瘦子重訓適合大磅數訓練嗎
第2342頁
title=  [問題] 大家有沒有發現,鐵克的訓練與動作...
第2342頁
title=  Re: [討論] 瘦子重訓適合大磅數訓練嗎
第2342頁
title=  [問題] 喝乳清蛋白會造成頭暈嗎??
第2342頁
title=  Re: [問題] 在家該怎練?
第2342頁
title=  Re: [問題] 降情況要繼續 重訓嗎
第2342頁
title=  [問題] 想請問加州價格
第2342頁
title=  [問題] 關於窄肩膀的問題
第2342頁
title=  Re: [問題] 關於窄肩膀的問題
第2343頁
title=  [情報] 台大新體徵求重訓室指導員一名
第2343頁
title=  [問題] 坐式機械飛鳥的問題??麻煩大家了
第2343頁
title=  [問題] 左右手差很多!!
第2343頁
title=  [問題] 有人要一起合購新竹泰克巴帝的日曬卷嗎？
第2343頁
title=  Re: [情報] 新大專先生產生
第2343頁
title=  Re: [請益]關於亞力的爭議款
第2343頁
title=  Re: [閒聊] 健身房的討厭鬼
第2343頁
title=  Re: [閒聊] 健身房的討厭鬼
第2343頁
title=  [問題] 體脂肪一天內暴增
第2343頁
title=  [問題] 請問有一個月繳錢的嗎
第2343頁
title=  [問題] 下腹部
第2343頁
title=  [心得] 如果要減肥，好像先有氧後重訓比較好
第2343頁
title=  [請益] 鮪魚肚怎麼消
第2343頁
title=  [問題] 請問養握起做機器
第2343頁
title=  補充蛋白質
第2343頁
title=  [請益] 我的象腿該怎麼變小
第2343頁
title=  Re: [問題] 在家該怎練?
第2343頁
title=  [問題] 健身房請假問題
第2343頁
title=  [請益] 我的菜單，請指教
第2343頁
title=  R

第2355頁
title=  [問題] 打排球的人的重訓??
第2355頁
title=  [問題] 請問新店世界健身俱樂部專業教練
第2355頁
title=  [問題] 請問有氧運動..
第2355頁
title=  [情報] 有人需要台南巨力屋的日曬卷嗎
第2355頁
title=  [問題] 內湖地區健身房的收費方式與費用
第2355頁
title=  [問題] 伏地挺身練胸肌(練肌肉振點-poppin)
第2355頁
title=  Re: [問題] 內湖地區健身房的收費方式與費用
第2355頁
title=  [請益]如何練成八快塊肌
第2355頁
title=  [心得] 敲膽經+瘦
第2355頁
title=  [分享] 街籃版看到的相簿
第2355頁
title=  Re: [請益]如何練成八快塊肌
第2355頁
title=  [問題] 怎麼練到手很不輸胡
第2355頁
title=  [問題] 去年曬出來
第2355頁
title=  [請益]跑完跑步機後膝蓋怎麼有條筋一直在跳!!?
第2355頁
title=  [問題] 加州三年的價格
第2355頁
title=  [問題] 乳清蛋白使用後...
第2355頁
title=  [問題] 喝豆漿會長胸部??
第2355頁
title=  [問題] 請問代謝提昇 也較容易餓嗎？
第2355頁
title=  [問題] 新手的請教
第2356頁
title=  [心得] Leviathan Reloaded V.S. BSN Atro-Phex
第2356頁
title=  [心得] 國外網站BodyBuilding.com 購買大絕招
第2356頁
title=  Re: [問題] 用滾輪訓練腹肌
第2356頁
title=  [討論] 蛋白質的價格與準備（個人心得）
第2356頁
title=  [問題] 要怎麼練下面？高手敎一下
沒這東西，只好割捨
第2356頁
title=  [問題] 精華區的偉德法則
第2356頁
title=  [討論] 大家最喜歡自己的那部位
第2356頁
title=  [問題] 用臥推練內胸?
第2356頁
title=  [心得] 雕塑腰部線條及大腿曲線
第2356頁
title=  [閒聊] 這是我，供大家參考。(內詳)
第2356頁
title=  [請益

In [4]:
#將資料存成檔案
df.to_excel("E:/Python 3.7/pyetl/Demodb0103/gym/ptt_3.xlsx",engine='xlsxwriter')

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://i.imgur.com/n0XUEz0.jpg
http://i.imgur.com/6OTdy2h.jpg
http://i.imgur.com/6s0ir33.jpg
http://i.imgur.com/VBnqArB.jpg
http://i.imgur.com/Ko2T0eC.jpg
http://i.imgur.com/tEja2YA.jpg
2018-07-03%2017:23
http://i.imgur.com/06JwyPe.jpg
http://i.imgur.com/bbyFBYw.jpg
http://i.imgur.com/Vc3057K.jpg
〔記者黃昭國／綜合報導〕玉山國家公園的排雲山莊最近整理環境，要將近80顆、每顆75
公斤的太陽能廢鉛酸電池運下山，承攬這項高山運務的原住民伍玉龍廣發英雄帖，邀請健
腳或大力士來接受電瓶超人的挑戰，每運一顆可獲得8000元代價，但往返塔塔加登山口至
排雲山莊長達17公里，通告一出引來熱議或獻策，但更多網友坦承「有命賺，可能沒命花
」。

伍玉龍在個人臉書上徵人，他說，徵求勇士把排雲山莊75公斤的鉛電池背回到塔塔加的警
察隊後面，運送一顆8000元，一人或多人合作、一天或二天揹下山都可以，下山當天發放
現金，個人累計運送下山達八顆加發獎金1萬元，運完為止。這篇廣徵英雄上山搬電池的
邀請文，受到山友及網友熱烈回應，質疑為何不用直升機載下山比較快，要用人力背下山
，簡直折磨人。

曾經在軍中當砲兵的網友說，軍中每一阿兵哥背運砲彈的重量都不超過50公斤，超重都用
車輛運送，網友認為，玉管處給的錢太少了，按照海拔高度與運送里程計算，合理的工資
應該是每公斤200元，一顆電池運下山應給15000元。更有人建議，何不把電池液體倒出來
分別包裝帶下山，才不會太重把人壓傷「有命賺錢、沒命享用」。

這篇徵人啟事，投書者幾乎沒人自告奮勇願意去當電瓶超人，但有人推薦說，嘉明湖有一
名協作可背100公斤，可以找他來幫忙，也有人說兩人用棍子扛下山，1人分4千元也不錯
。更有人建議用科學方法解決